In [20]:
import sys
import os

# Assuming the text_prediction module is in the 'src' directory
module_path = "/Users/aadil/Documents/Workspace/ml-projects/wiki-llm/src/"  # Replace with the correct path to your 'src' directory
if module_path not in sys.path:
    sys.path.append(module_path)

In [111]:
import torch
from torch.utils.data import DataLoader, DistributedSampler, Dataset as TDataset
from torch.nn.utils.rnn import pad_sequence
from datasets import load_dataset, load_from_disk, Dataset
import os
import pyarrow as pa
from torch.utils.data import Dataset as TorchDataset
import logging
import random
import re

from text_prediction.utils import RankFilter

class TokenizedDataset(TorchDataset):
    def __init__(self, tokenized_dataset, block_size, device="cpu"):
        """
        tokenized_examples: List of tokenized sequences (each a list of token IDs).
        block_size: Length of each input sequence.
        """
        self.device = device
        self.block_size = block_size

        self._data = [torch.tensor(example['input_ids'], dtype=torch.long) for example in tokenized_dataset if len(example['input_ids']) > block_size]
        
    def __len__(self):
        return len(self._data) - self.block_size  # Max index to sample from

    def __getitem__(self, idx):
        """
        Return a single sample (input and target sequences)
        """
        example = self._data[idx]
        ix = torch.randint(0, len(example) - self.block_size, (1,)).item()
        input_seq = example[ix:ix + self.block_size]
        label_seq = example[ix + 1:ix + self.block_size + 1]
        return input_seq.to(self.device), label_seq.to(self.device)

class DataPipeline:

    def __init__(self, tokenizer, max_len, block_size, regenerate=False, num_samples=10000, verbose=False, augment_data=False, parent_path="."):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.block_size = block_size
        self.regenerate = regenerate
        self.num_samples = num_samples
        self.train_dataloader = None
        self.val_dataloader = None
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.verbose = verbose
        self.augment_data = augment_data
        self.parent_path = parent_path

        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')  # Update logging format
        self.logger = logging.getLogger(__name__)
        self.logger.addFilter(RankFilter(0))
        self.tokenizer.add_special_tokens({"additional_special_tokens": ["<ARTICLE_START>", "<ARTICLE_END>"]})

    def log(self, message, level=logging.INFO):
        if self.verbose:
            self.logger.log(level, message)

    def _sanitize_text(self, text):
        """Sanitizes the text while preserving meaningful dashes, numerical values with units, and hyperlink text."""
        
        # Preserve Markdown-style and HTML links by keeping only the text
        text = re.sub(r'\[([^\]]+)\]\(http\S+\)', r'\1', text)  # Handles Markdown links
        text = re.sub(r'<a\s+href=["\']http\S+["\']>(.*?)</a>', r'\1', text, flags=re.IGNORECASE)  # Handles HTML links

        # Remove standalone URLs
        text = re.sub(r'http\S+|www\S+', '', text, flags=re.MULTILINE)

        # Ensure we preserve dashes in hyphenated place names (e.g., Indianapolis–Carmel–Anderson)
        text = re.sub(r'(\w)\s*[-–]\s*(\w)', r'\1–\2', text)  # Normalize hyphens and remove unwanted spaces around them

        # Keep valid numerical values with units (e.g., "3.0 square miles (7.8 km2)")
        text = re.sub(r'(\d+(\.\d+)?)\s*([a-zA-Z²]+)', r'\1 \3', text)  # Ensures numbers and units stay together
        text = re.sub(r'\((\d+(\.\d+)?\s*[a-zA-Z²]+)\)', r'(\1)', text)  # Ensures parenthetical units remain intact

        # Preserve valid year ranges (e.g., 1992-2002)
        text = re.sub(r'(?<!\d)(\d{4})-(\d{4})(?!\d)', r'\1-\2', text)  # Ensure valid formatting

        # Remove unwanted characters but keep punctuation, parentheses, and percentage signs
        text = re.sub(r'[^a-zA-Z0-9\s.,!?\'\"()%-²]', '', text)

        # Normalize spaces
        text = re.sub(r'\s+', ' ', text).strip()

        return text


    def _tokenize_function(self, examples):
        """Tokenizes the examples, adds special tokens, and optionally augments data."""
        
        # Sanitize text data
        sanitized_texts = [self._sanitize_text(text) for text in examples["text"]]

        # Add special tokens to the beginning and end of each text
        texts_with_special_tokens = [
            "<ARTICLE_START>" + text + "<ARTICLE_END>" for text in sanitized_texts
        ]

        tokenized = self.tokenizer(
            texts_with_special_tokens,
            truncation=True,
            max_length=self.max_len,
            add_special_tokens=False,
        )

        if self.augment_data:
            # Add data augmentation logic here
            tokenized["input_ids"] = self._augment(tokenized["input_ids"])

        return tokenized

    def _augment(self, input_ids):
        # Implement data augmentation logic
        # For example, randomly mask some tokens
        augmented = []
        for ids in input_ids:
            if random.random() < 0.1:  # 10% chance to mask a token
                ids[random.randint(0, len(ids) - 1)] = self.tokenizer.mask_token_id
            augmented.append(ids)
        return augmented

    def _get_tokenized_dataset(self):
        tokenized_dataset_path = f"{self.parent_path}/data/{self.tokenizer.name_or_path}/wiki/tokenized_augmented" if self.augment_data else f"{self.parent_path}/data/{self.tokenizer.name_or_path}/wiki/tokenized"

        if not os.path.exists(tokenized_dataset_path) or self.regenerate:
            self.log("Local cache of dataset not found, downloading and tokenizing dataset...")
            # Load dataset (small subset of num_samples samples)
            ds = load_dataset("wikimedia/wikipedia", "20231101.en", split="train")
            ds = ds.select(range(self.num_samples))
            # Select only the 'text' column
            ds = ds.remove_columns([col for col in ds.column_names if col != "text"])
            # Tokenize the dataset
            ds = ds.map(self._tokenize_function, batched=True)
            ds.save_to_disk(tokenized_dataset_path)
        else:
            self.log("Local cache of dataset found, loading tokenized dataset...")
            ds = load_from_disk(tokenized_dataset_path)
        return ds

    def _is_dataset_valid(self, dataset_path):
        try:
            ds = load_from_disk(dataset_path)
            # Attempt to read a small portion of the dataset to ensure it's valid
            _ = ds[:1]
            return True
        except (pa.lib.ArrowInvalid, FileNotFoundError):
            return False

    @staticmethod
    def custom_collate(batch, tokenizer):
        inputs, labels = zip(*batch)
        pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
        inputs = pad_sequence(inputs, batch_first=True, padding_value=pad_token_id)
        labels = pad_sequence(labels, batch_first=True, padding_value=pad_token_id)

        # Create attention masks
        B, T = inputs.shape
        tril_mask = torch.tril(torch.ones(T, T)).bool().to(inputs.device) #move tril to the same device as inputs
        attention_mask = (inputs != pad_token_id).unsqueeze(1).repeat(1, T, 1) & tril_mask #create a causal mask.
        return inputs, labels, attention_mask.to(inputs.device)

    def get_dataloader(self, batch_size, split="train", shuffle=True):
        # Ensure split is either "train" or "val"
        if split not in ["train", "val"]:
            raise ValueError("split must be either 'train' or 'val'")
        
        # DataLoader with random sampling
        tds = self._get_tokenized_dataset()
        
        # Split the dataset into train and validation sets
        split_datasets = tds.train_test_split(test_size=0.2, seed=42)
        selected_dataset = split_datasets["train" if split == "train" else "test"]
        
        dataset = TokenizedDataset(selected_dataset, self.block_size, self.device)
        
        return DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=shuffle,
            collate_fn=lambda batch: DataPipeline.custom_collate(batch, self.tokenizer),
        )

In [112]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
dp = DataPipeline(tokenizer, max_len=512, block_size=128, regenerate=False, num_samples=10000, verbose=True, augment_data=False, parent_path="..")

In [114]:
train_loader = dp.get_dataloader(32)

# Get a batch
i = 0
for x, y, _ in train_loader:
    print(f"example {i} inputs")

    print(tokenizer.decode(x[0]))
    print(f"example {i} labels")
    print(tokenizer.decode(y[0]))
    i += 1
    if i > 5:
        break

2025-03-20 08:33:11,571 - Local cache of dataset not found, downloading and tokenizing dataset...
Saving the dataset (1/1 shards): 100%|██████████| 10000/10000 [00:00<00:00, 46121.41 examples/s]


example 0 inputs
 played eight seasons for the Denver Broncos from 1996 through 2003 in the National Football League (NFL). He is the cousin of former NBA player Cuttino Mobley. Biography One of nine children born to parents who divorced when he was thirteen years old, Mobley lived with his father until the age of sixteen. After his father suffered a stroke, Mobley moved in with his mother, who demanded he leave high school to help support his family. Mobley spent a year living on the street in an old car before a friend's family took him in. Lacking the academic credentials for a Division I school, Mobley went on to
example 0 labels
 eight seasons for the Denver Broncos from 1996 through 2003 in the National Football League (NFL). He is the cousin of former NBA player Cuttino Mobley. Biography One of nine children born to parents who divorced when he was thirteen years old, Mobley lived with his father until the age of sixteen. After his father suffered a stroke, Mobley moved in with 

In [7]:
import text_prediction

ModuleNotFoundError: No module named 'text_prediction'

In [8]:
import sys
!{sys.executable} -m pip install --no-cache-dir text_prediction

In [1]:
import sys
import os

# Assuming the text_prediction module is in the 'src' directory
module_path = "/Users/aadil/Documents/Workspace/ml-projects/wiki-llm/src/"  # Replace with the correct path to your 'src' directory
if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
from text_prediction.data_pipeline import DataPipeline

/Users/aadil/Documents/Workspace/ml-projects/wiki-llm/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

dp = DataPipeline(
    tokenizer=tokenizer,
    max_len=512,
    block_size=128,
    num_samples=10000,
    verbose=True,
    augment_data=False,
    parent_path=".."
)

In [4]:
dl = dp.get_dataloader(batch_size=32, shuffle=True)

2025-03-18 23:14:19,030 - Local cache of dataset found, loading tokenized dataset...


getting tokenized dataset


In [9]:
for i in range(5):
    x, y = next(iter(dl))
    print(f"Batch {i + 1}")
    print("Input shape:", x.shape)
    print("Label shape:", y.shape)
    print(f"Input sequence: {tokenizer.decode(x[0].tolist())}")
    print(f"Output sequence: {tokenizer.decode(y[0].tolist())}")

Batch 1
Input shape: torch.Size([32, 128])
Label shape: torch.Size([32, 128])
Input sequence: : Army, Navy and Aerospace Force (Title VII, chapter VII, Art. 217)

This is a subtle yet important distinction, both in terms of emphasizing the civil nature of the National Police, but also adapting the national police to function as a paramilitary force which can perform military duties as a result of the Colombian Conflict. This has led to some of the most important police units adopting military training and conducting special operations alongside the Colombian Army, Aerospace Force, and Navy. ThereforeThe history of Colombia includes the settlements and society by indigenous peoples, most notably, the Muisca Confederation, Quimbaya Civilization, and Tairona Chief
Output sequence:  Army, Navy and Aerospace Force (Title VII, chapter VII, Art. 217)

This is a subtle yet important distinction, both in terms of emphasizing the civil nature of the National Police, but also adapting the nationa